In [ ]:
from cedne import utils

In [ ]:
w = utils.makeWorm()
nn = w.networks["Neutral"]
# utils.loadNeuropeptides(nn)
utils.loadNeurotransmitters(nn)
utils.loadGapJunctions(nn)

# Get all inputs to a neuron.

In [ ]:
## Shell graph here.
center = nn.neurons['ASEL']
sub_edges = center.get_connections()
nn_sub = nn.subnetwork(connections=sub_edges)

utils.plot_shell(nn_sub, center=center, figsize=(6,6), fontsize=10)

# Get all paths between two neurons.

## Print all direct paths between two neurons. This could be useful for circuit analysis experiments or to generate molecular screens.

In [ ]:
n1 = nn.neurons['AWCR']
n2 = nn.neurons['ASEL']
print("||Neuron 1|| Neuron 2|| Connection type || Putative connection details||")
print("=================================")
edge_labels={}
for e,c in n1.get_connections(n2).items():
    if c.edge_type == 'chemical-synapse':
        print(f"||{c.pre_neuron.name} || {c.post_neuron.name} || {c.edge_type} || " + f"{c.putative_neurotrasmitter_receptors} ||")
    elif c.edge_type == 'gap-junction':
        print(f"||{c.pre_neuron.name} || {c.post_neuron.name} || {c.edge_type} || " + f"{c.putative_gapjn_subunits} ||")
    else:
        print(f"||{c.pre_neuron.name} || {c.post_neuron.name} || {c.edge_type} ||" )
    edge_labels[e] = c.putative_neurotrasmitter_receptors
    # if e[0].name in nn.neurons and e[1].name in nn.neurons and c.edge_type == 'chemical-synapse':
    #     print(f"{c.putative_connection}")

In [ ]:
center = [n1, n2]
path_length = 1
conn_path = n1.paths_to(n2, path_length=path_length) + n2.paths_to(n1, path_length=path_length)
all_steps_on_path = [conn_._id for conn in conn_path for conn_ in conn]
connection_subnetwork = nn.subnetwork(connections=all_steps_on_path)
utils.plot_shell(connection_subnetwork, center=center, figsize=(1,3), fontsize=10)

## Print paths that are at most N steps removed.

In [ ]:
center = [n1, n2]
path_length = 2
conn_path = n1.paths_to(n2, path_length=path_length) + n2.paths_to(n1, path_length=path_length)
all_steps_on_path = [conn_._id for conn in conn_path for conn_ in conn]
connection_subnetwork = nn.subnetwork(connections=all_steps_on_path)
utils.plot_shell(connection_subnetwork, center=center, figsize=(6,6), fontsize=10)

In [ ]:
## Make a matrix of connections vs connection types and do a PCA analysis to see if there are groups of connections across
## the matrix

In [ ]:
all_NPs = {n:[] for n in nn.neurons.keys()}
for e,c in nn.connections.items():
    if c.edge_type not in ['chemical-synapse', 'gap-junction']:
        all_NPs[e[0].name].append(c.edge_type)

count_NPs = {n:[] for n in nn.neurons.keys()}
for n in all_NPs:
    all_NPs[n] = set(all_NPs[n])
    count_NPs[n] = len(all_NPs[n])

In [ ]:
print(dict(sorted(count_NPs.items(), key=lambda item: item[1], reverse=True)))